In [31]:
import numpy as np
import matplotlib

In [21]:
def generateData(d, N, noise=0.1):
    np.random.seed(12345)
    X = np.concatenate([np.ones([N, 1]), np.random.randn(N, d)], axis=1)
    lambdaTrue = -1 + 2 * np.tan(np.random.randn(len(X[0])))
    Y = 1 * ((X @ lambdaTrue + noise * np.random.randn(N)) > 0)
    D = X, Y
    return D

In [22]:
def split(D, train_size):
    x,y=D
    x_trainning=[]
    x_testing=[]
    y_traning=[]
    y_testing=[]
    for i in range(0,train_size):
        x_trainning.append(x[i])
        y_traning.append(y[i])
    for i in range (train_size,len(y)):
        x_testing.append(x[i])
        y_testing.append(y[i])
    return x_trainning,x_testing,y_traning,y_testing

print(split(generateData(10,100),50))

([array([ 1.        , -0.20470766,  0.47894334, -0.51943872, -0.5557303 ,
        1.96578057,  1.39340583,  0.09290788,  0.28174615,  0.76902257,
        1.24643474]), array([ 1.        ,  1.00718936, -1.29622111,  0.27499163,  0.22891288,
        1.35291684,  0.88642934, -2.00163731, -0.37184254,  1.66902531,
       -0.43856974]), array([ 1.00000000e+00, -5.39741446e-01,  4.76985010e-01,  3.24894392e+00,
       -1.02122752e+00, -5.77087303e-01,  1.24121276e-01,  3.02613562e-01,
        5.23772068e-01,  9.40277775e-04,  1.34380979e+00]), array([ 1.        , -0.71354399, -0.83115354, -2.37023165, -1.86076079,
       -0.8607574 ,  0.56014529, -1.26593449,  0.11982712, -1.06351245,
        0.33288272]), array([ 1.        , -2.35941881, -0.19954296, -1.54199553, -0.97073591,
       -1.30703025,  0.28634975,  0.37798411, -0.75388653,  0.33128565,
        1.34974221]), array([ 1.        ,  0.06987669,  0.24667411, -0.0118616 ,  1.00481159,
        1.32719461, -0.91926156, -1.54910644,  0.022

In [23]:
def model(x, par):
    z = x @ par
    # return 1 / (np.exp(-z)+1)
    return np.power(1 / (1 + np.power(np.e, -z)), -1)

In [29]:
def dModel(x, par):
    f = np.power(1 + np.power(np.e, -np.dot(x, par)), -1)
    gradient = f * (1 - f) * par
    return gradient

In [30]:
def objective(par, data):
    ell=0
    X, Y = data
    for n in range(0,len(X)):
        x, y = X[n], Y[n]
        # f = np.power((np.dot(par,x)),np.e)
        f=model(X,par)
        if y==1:
            f=(f)
        else:
            deltat=(1-f)
        s =-(np.log(f))
        ell = ell + s
    return ell

In [33]:
def gradient(par, data):
    grad = np.zeros(len(par))
    X, Y = data
    for n in range (0,(X)):
        f = model(X, par)
        x, y = X[n], Y[n]
        deltaf = dModel(X,par)
        # s = -(y-f)*deltaf/(y*f+(1-y)*(1-f)) #not sure if deltaf should be here
        s = -(y-f)*dModel(X,par)/(y*f+(1-y)*(1-f))
        grad = grad + s
    return grad

In [34]:
def train(par0, eta, T, data):
    par = par0
    obj =[ ]
    for t in range (0,T):
        ell = objective(par,data)
        obj.append(ell)
        grad = gradient(par,data)
        par = par - grad*ell
    return par, obj